For this project, I am going to scrape the www.cfbstats.com site for the PAC-12 college football conference statistics.  I will perform two separate scrapes and then combine the two into a single DataFrame.  

First I need to load the necessary modules.

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd

I am going to start with the conference statistics.  

# Conference Stats

In [2]:
url='http://www.cfbstats.com/2019/leader/905/team/offense/split01/category09/sort01.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Name"
3:"G"
4:"TD"
5:"FG"
6:"1XP"
7:"2XP"
8:"Safety"
9:"Points"
10:"Points/G"


This code chunk pulled out the column headers.  I knew there were 10 columns so that I why the line of code `if len(T)!=10` is set at that value.  You will see later on that I have to change that value for the individual team data scrapes.  

Having done this, we are now ready to create the DataFrame with the scraped table values.

In [3]:
dict_conf={title:column for (title,column) in col}
df_conf=pd.DataFrame(dict_conf)
df_conf

,,Name,G,TD,FG,1XP,2XP,Safety,Points,Points/G
0,1,Washington State,8,42,13,41,1,0,334,41.8
1,2,Oregon,8,39,5,37,1,0,288,36.0
2,3,Washington,8,34,15,33,1,1,286,35.8
3,4,Utah,8,35,7,34,0,0,265,33.1
4,5,Arizona,8,34,6,32,1,1,258,32.3
5,6,USC,8,32,8,32,0,1,250,31.3
6,7,Oregon State,7,30,2,27,0,0,213,30.4
7,8,UCLA,8,31,5,30,1,0,233,29.1
8,9,Colorado,8,26,12,25,0,0,217,27.1
9,10,Arizona State,8,21,13,20,1,0,187,23.4


That worked perfectly.  We do, however, have a redundant column that I will go ahead and drop.

In [4]:
df_conf = df_conf.drop([df_conf.columns[0]],  axis='columns')
df_conf

,Name,G,TD,FG,1XP,2XP,Safety,Points,Points/G
0,Washington State,8,42,13,41,1,0,334,41.8
1,Oregon,8,39,5,37,1,0,288,36.0
2,Washington,8,34,15,33,1,1,286,35.8
3,Utah,8,35,7,34,0,0,265,33.1
4,Arizona,8,34,6,32,1,1,258,32.3
5,USC,8,32,8,32,0,1,250,31.3
6,Oregon State,7,30,2,27,0,0,213,30.4
7,UCLA,8,31,5,30,1,0,233,29.1
8,Colorado,8,26,12,25,0,0,217,27.1
9,Arizona State,8,21,13,20,1,0,187,23.4


Okay, our first scrape was successful and we have our first DataFrame of the conference statistics.  Because there are only nine variables, I will do another scrape for each team within the conference.

# Team Stats

I tried using a looping function that would scrape for each team all at once, but I couldn't ever get it to work.  I spent hours trying to figure it out but ultimately had to just repeat the process separately for each team.  Because of this, I will not provide explanations for each team since they are all the same.  I will just provide them for the first team, Utah Utes.

**Utah Utes**

In [5]:
url='http://www.cfbstats.com/2019/team/732/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Utah"
3:"Opponents"


As I mentioned earlier, I knew that I would have to change the `if len(T)!=3` section to the correct value for this table.  

One interesting thing about this particular scrape is that the data are set up differently than what we are used to.  The data are transposed, meaning that the variables that are normally columns are set up as rows (see image below).  This isn't a big deal because we can still scrape the data and then perform a transpose function to orient things correctly.

In [6]:
from IPython.display import Image
img = 'screenshot1.png'
Image(url=img)

In [7]:
dict_utah={title:column for (title,column) in col}
df_utah=pd.DataFrame(dict_utah)
df_utah

,,Utah,Opponents
0,Scoring: Points/Game,33.1,10.3
1,Scoring: Games - Points,8 - 265,8 - 82
2,First Downs: Total,185,101
3,First Downs: Rushing - Passing - By Penalty,92 - 75 - 18,29 - 60 - 12
4,Rushing: Yards / Attempt,5.07,2.45
5,Rushing: Attempts - Yards - TD,346 - 1755 - 21,184 - 451 - 3
6,Passing: Rating,175.61,103.66
7,Passing: Yards,1864,1397
8,Passing: Attempts - Completions - Interceptio...,185 - 134 - 1 - 11,250 - 140 - 9 - 6
9,Total Offense: Yards / Play,6.82,4.26


Our scrape was successful.  

I am not interested in the "Opponent" column, so I will drop that.  I will also perform a transform to the DataFrame to make the rows into columns.  

Also, when I did the transpose, it made my header column appear as the first row of data.  Therefore, I need to delete that row and turn it into a header.

In [8]:
df_utah = df_utah.drop(columns=['Opponents']).T
header = df_utah.iloc[0]
df_utah = df_utah[1:]
df_utah = df_utah.rename(columns = header)
df_utah

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Utah,33.1,8 - 265,185,92 - 75 - 18,5.07,346 - 1755 - 21,175.61,1864,185 - 134 - 1 - 11,6.82,...,64.29%,14 - 9,80.56%,36 - 29,63.6%,11 - 7,97.1%,35 - 34,-,0 - 0


Okay, after a successful scrape and many data transformation/clean-up steps, we have generated another DataFrame for an individual PAC-12 team, the Utah Utes.  Here are the transformation/clean-up steps we performed: dropped an unnecessary column, transposed the data from rows to columns, and assigned an appropriate column header.  

This process will be repeated for each individual PAC-12 team.

**Washington State Cougars**

In [9]:
url='http://www.cfbstats.com/2019/team/754/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Washington State"
3:"Opponents"


In [10]:
dict_wash_st={title:column for (title,column) in col}
df_wash_st=pd.DataFrame(dict_wash_st)
df_wash_st

,,Washington State,Opponents
0,Scoring: Points/Game,41.8,29.8
1,Scoring: Games - Points,8 - 334,8 - 238
2,First Downs: Total,202,186
3,First Downs: Rushing - Passing - By Penalty,36 - 154 - 12,82 - 87 - 17
4,Rushing: Yards / Attempt,5.10,4.97
5,Rushing: Attempts - Yards - TD,134 - 684 - 8,311 - 1547 - 16
6,Passing: Rating,162.07,142.77
7,Passing: Yards,3491,2055
8,Passing: Attempts - Completions - Interceptio...,418 - 290 - 9 - 34,250 - 160 - 6 - 11
9,Total Offense: Yards / Play,7.56,6.42


In [11]:
df_wash_st = df_wash_st.drop(columns=['Opponents']).T
header = df_wash_st.iloc[0]
df_wash_st = df_wash_st[1:]
df_wash_st = df_wash_st.rename(columns = header)
df_wash_st

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Washington State,41.8,8 - 334,202,36 - 154 - 12,5.10,134 - 684 - 8,162.07,3491,418 - 290 - 9 - 34,7.56,...,72.73%,11 - 8,92.68%,41 - 38,100%,13 - 13,100%,41 - 41,100%,1 - 1


**Oregon Ducks**

In [12]:
url='http://www.cfbstats.com/2019/team/529/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Oregon"
3:"Opponents"


In [13]:
dict_ore={title:column for (title,column) in col}
df_ore=pd.DataFrame(dict_ore)
df_ore

,,Oregon,Opponents
0,Scoring: Points/Game,36.0,14.8
1,Scoring: Games - Points,8 - 288,8 - 118
2,First Downs: Total,193,140
3,First Downs: Rushing - Passing - By Penalty,72 - 107 - 14,48 - 77 - 15
4,Rushing: Yards / Attempt,5.05,3.16
5,Rushing: Attempts - Yards - TD,301 - 1521 - 14,256 - 810 - 3
6,Passing: Rating,162.43,107.05
7,Passing: Yards,2207,1656
8,Passing: Attempts - Completions - Interceptio...,276 - 189 - 1 - 23,281 - 160 - 14 - 9
9,Total Offense: Yards / Play,6.46,4.59


In [14]:
df_ore = df_ore.drop(columns=['Opponents']).T
header = df_ore.iloc[0]
df_ore = df_ore[1:]
df_ore = df_ore.rename(columns = header)
df_ore

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Oregon,36.0,8 - 288,193,72 - 107 - 14,5.05,301 - 1521 - 14,162.43,2207,276 - 189 - 1 - 23,6.46,...,50%,18 - 9,80.49%,41 - 33,62.5%,8 - 5,97.4%,38 - 37,100%,1 - 1


**Washington Huskies**

In [15]:
url='http://www.cfbstats.com/2019/team/756/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Washington"
3:"Opponents"


In [16]:
dict_wash={title:column for (title,column) in col}
df_wash=pd.DataFrame(dict_wash)
df_wash

,,Washington,Opponents
0,Scoring: Points/Game,35.8,21.5
1,Scoring: Games - Points,8 - 286,8 - 172
2,First Downs: Total,168,168
3,First Downs: Rushing - Passing - By Penalty,75 - 83 - 10,68 - 85 - 15
4,Rushing: Yards / Attempt,4.66,4.19
5,Rushing: Attempts - Yards - TD,295 - 1376 - 15,282 - 1181 - 11
6,Passing: Rating,158.39,119.86
7,Passing: Yards,2000,1798
8,Passing: Attempts - Completions - Interceptio...,236 - 159 - 3 - 16,277 - 164 - 8 - 10
9,Total Offense: Yards / Play,6.36,5.33


In [17]:
df_wash = df_wash.drop(columns=['Opponents']).T
header = df_wash.iloc[0]
df_wash = df_wash[1:]
df_wash = df_wash.rename(columns = header)
df_wash

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Washington,35.8,8 - 286,168,75 - 83 - 10,4.66,295 - 1376 - 15,158.39,2000,236 - 159 - 3 - 16,6.36,...,62.5%,16 - 10,91.43%,35 - 32,100%,15 - 15,100%,33 - 33,100%,1 - 1


**Arizona Wildcats**

In [18]:
url='http://www.cfbstats.com/2019/team/29/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Arizona"
3:"Opponents"


In [19]:
dict_arz={title:column for (title,column) in col}
df_arz=pd.DataFrame(dict_arz)
df_arz

,,Arizona,Opponents
0,Scoring: Points/Game,32.3,35.0
1,Scoring: Games - Points,8 - 258,8 - 280
2,First Downs: Total,189,190
3,First Downs: Rushing - Passing - By Penalty,82 - 101 - 6,68 - 109 - 13
4,Rushing: Yards / Attempt,5.24,4.64
5,Rushing: Attempts - Yards - TD,309 - 1620 - 15,273 - 1266 - 15
6,Passing: Rating,147.45,138.37
7,Passing: Yards,2316,2493
8,Passing: Attempts - Completions - Interceptio...,285 - 181 - 9 - 19,333 - 210 - 9 - 18
9,Total Offense: Yards / Play,6.63,6.20


In [20]:
df_arz = df_arz.drop(columns=['Opponents']).T
header = df_arz.iloc[0]
df_arz = df_arz[1:]
df_arz = df_arz.rename(columns = header)
df_arz

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Arizona,32.3,8 - 258,189,82 - 101 - 6,5.24,309 - 1620 - 15,147.45,2316,285 - 181 - 9 - 19,6.63,...,81.82%,11 - 9,88%,25 - 22,60%,10 - 6,97%,33 - 32,100%,1 - 1


**USC Trojans**

In [21]:
url='http://www.cfbstats.com/2019/team/657/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"USC"
3:"Opponents"


In [22]:
dict_usc={title:column for (title,column) in col}
df_usc=pd.DataFrame(dict_usc)
df_usc

,,USC,Opponents
0,Scoring: Points/Game,31.3,24.9
1,Scoring: Games - Points,8 - 250,8 - 199
2,First Downs: Total,175,181
3,First Downs: Rushing - Passing - By Penalty,64 - 102 - 9,83 - 82 - 16
4,Rushing: Yards / Attempt,4.48,4.84
5,Rushing: Attempts - Yards - TD,261 - 1170 - 13,314 - 1521 - 10
6,Passing: Rating,155.46,128.90
7,Passing: Yards,2369,1914
8,Passing: Attempts - Completions - Interceptio...,282 - 200 - 10 - 18,277 - 166 - 3 - 11
9,Total Offense: Yards / Play,6.52,5.81


In [23]:
df_usc = df_usc.drop(columns=['Opponents']).T
header = df_usc.iloc[0]
df_usc = df_usc[1:]
df_usc = df_usc.rename(columns = header)
df_usc

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
USC,31.3,8 - 250,175,64 - 102 - 9,4.48,261 - 1170 - 13,155.46,2369,282 - 200 - 10 - 18,6.52,...,66.67%,6 - 4,77.42%,31 - 24,88.9%,9 - 8,100%,32 - 32,-,0 - 0


**Oregon State Beavers**

In [24]:
url='http://www.cfbstats.com/2019/team/528/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Oregon State"
3:"Opponents"


In [25]:
dict_orst={title:column for (title,column) in col}
df_orst=pd.DataFrame(dict_orst)
df_orst

,,Oregon State,Opponents
0,Scoring: Points/Game,30.4,31.6
1,Scoring: Games - Points,7 - 213,7 - 221
2,First Downs: Total,160,165
3,First Downs: Rushing - Passing - By Penalty,69 - 82 - 9,73 - 75 - 17
4,Rushing: Yards / Attempt,4.88,4.48
5,Rushing: Attempts - Yards - TD,240 - 1170 - 12,299 - 1341 - 12
6,Passing: Rating,138.28,152.76
7,Passing: Yards,1706,1647
8,Passing: Attempts - Completions - Interceptio...,250 - 147 - 2 - 18,205 - 128 - 3 - 16
9,Total Offense: Yards / Play,5.87,5.93


In [26]:
df_orst = df_orst.drop(columns=['Opponents']).T
header = df_orst.iloc[0]
df_orst = df_orst[1:]
df_orst = df_orst.rename(columns = header)
df_orst

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Oregon State,30.4,7 - 213,160,69 - 82 - 9,4.88,240 - 1170 - 12,138.28,1706,250 - 147 - 2 - 18,5.87,...,71.43%,14 - 10,88.46%,26 - 23,28.6%,7 - 2,96.4%,28 - 27,0%,2 - 0


**UCLA Bruins**

In [27]:
url='http://www.cfbstats.com/2019/team/110/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"UCLA"
3:"Opponents"


In [28]:
dict_ucla={title:column for (title,column) in col}
df_ucla=pd.DataFrame(dict_ucla)
df_ucla

,,UCLA,Opponents
0,Scoring: Points/Game,29.1,34.3
1,Scoring: Games - Points,8 - 233,8 - 274
2,First Downs: Total,176,169
3,First Downs: Rushing - Passing - By Penalty,77 - 83 - 16,53 - 107 - 9
4,Rushing: Yards / Attempt,4.02,4.13
5,Rushing: Attempts - Yards - TD,333 - 1337 - 13,278 - 1147 - 10
6,Passing: Rating,134.34,168.87
7,Passing: Yards,1895,2454
8,Passing: Attempts - Completions - Interceptio...,265 - 158 - 7 - 16,272 - 180 - 3 - 24
9,Total Offense: Yards / Play,5.40,6.55


In [29]:
df_ucla = df_ucla.drop(columns=['Opponents']).T
header = df_ucla.iloc[0]
df_ucla = df_ucla[1:]
df_ucla = df_ucla.rename(columns = header)
df_ucla

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
UCLA,29.1,8 - 233,176,77 - 83 - 16,4.02,333 - 1337 - 13,134.34,1895,265 - 158 - 7 - 16,5.40,...,42.86%,21 - 9,81.82%,33 - 27,62.5%,8 - 5,100%,30 - 30,100%,1 - 1


**Colorado Buffaloes**

In [30]:
url='http://www.cfbstats.com/2019/team/157/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Colorado"
3:"Opponents"


In [31]:
dict_col={title:column for (title,column) in col}
df_col=pd.DataFrame(dict_col)
df_col

,,Colorado,Opponents
0,Scoring: Points/Game,27.1,34.9
1,Scoring: Games - Points,8 - 217,8 - 279
2,First Downs: Total,175,188
3,First Downs: Rushing - Passing - By Penalty,73 - 90 - 12,66 - 111 - 11
4,Rushing: Yards / Attempt,4.31,4.82
5,Rushing: Attempts - Yards - TD,296 - 1277 - 11,265 - 1277 - 15
6,Passing: Rating,133.69,158.08
7,Passing: Yards,2097,2618
8,Passing: Attempts - Completions - Interceptio...,289 - 182 - 9 - 14,297 - 193 - 8 - 22
9,Total Offense: Yards / Play,5.77,6.93


In [32]:
df_col = df_col.drop(columns=['Opponents']).T
header = df_col.iloc[0]
df_col = df_col[1:]
df_col = df_col.rename(columns = header)
df_col

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Colorado,27.1,8 - 217,175,73 - 90 - 12,4.31,296 - 1277 - 11,133.69,2097,289 - 182 - 9 - 14,5.77,...,61.54%,13 - 8,80.65%,31 - 25,80%,15 - 12,96.2%,26 - 25,-,0 - 0


**Arizona State Sun Devils**

In [33]:
url='http://www.cfbstats.com/2019/team/28/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Arizona State"
3:"Opponents"


In [34]:
dict_azst={title:column for (title,column) in col}
df_azst=pd.DataFrame(dict_azst)
df_azst

,,Arizona State,Opponents
0,Scoring: Points/Game,23.4,21.1
1,Scoring: Games - Points,8 - 187,8 - 169
2,First Downs: Total,138,159
3,First Downs: Rushing - Passing - By Penalty,53 - 77 - 8,54 - 89 - 16
4,Rushing: Yards / Attempt,3.69,3.22
5,Rushing: Attempts - Yards - TD,280 - 1033 - 10,285 - 918 - 11
6,Passing: Rating,146.67,132.13
7,Passing: Yards,1902,1944
8,Passing: Attempts - Completions - Interceptio...,223 - 135 - 2 - 11,282 - 179 - 3 - 11
9,Total Offense: Yards / Play,5.83,5.05


In [35]:
df_azst = df_azst.drop(columns=['Opponents']).T
header = df_azst.iloc[0]
df_azst = df_azst[1:]
df_azst = df_azst.rename(columns = header)
df_azst

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Arizona State,23.4,8 - 187,138,53 - 77 - 8,3.69,280 - 1033 - 10,146.67,1902,223 - 135 - 2 - 11,5.83,...,50%,10 - 5,92%,25 - 23,76.5%,17 - 13,100%,20 - 20,100%,1 - 1


**Stanford Cardinal**

In [36]:
url='http://www.cfbstats.com/2019/team/674/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"Stanford"
3:"Opponents"


In [37]:
dict_stan={title:column for (title,column) in col}
df_stan=pd.DataFrame(dict_stan)
df_stan

,,Stanford,Opponents
0,Scoring: Points/Game,22.6,28.0
1,Scoring: Games - Points,8 - 181,8 - 224
2,First Downs: Total,157,171
3,First Downs: Rushing - Passing - By Penalty,51 - 88 - 18,65 - 97 - 9
4,Rushing: Yards / Attempt,3.68,4.46
5,Rushing: Attempts - Yards - TD,259 - 954 - 6,270 - 1204 - 12
6,Passing: Rating,125.57,148.36
7,Passing: Yards,1834,2108
8,Passing: Attempts - Completions - Interceptio...,283 - 171 - 3 - 11,263 - 169 - 6 - 17
9,Total Offense: Yards / Play,5.14,6.21


In [38]:
df_stan = df_stan.drop(columns=['Opponents']).T
header = df_stan.iloc[0]
df_stan = df_stan[1:]
df_stan = df_stan.rename(columns = header)
df_stan

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Stanford,22.6,8 - 181,157,51 - 88 - 18,3.68,259 - 954 - 6,125.57,1834,283 - 171 - 3 - 11,5.14,...,30%,10 - 3,92.31%,26 - 24,77.8%,18 - 14,100%,19 - 19,0%,1 - 0


**California Golden Bears**

In [39]:
url='http://www.cfbstats.com/2019/team/107/index.html'#Create a handle, page, to handle the contents of the website
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

1:""
2:"California"
3:"Opponents"


In [40]:
dict_cal={title:column for (title,column) in col}
df_cal=pd.DataFrame(dict_cal)
df_cal

,,California,Opponents
0,Scoring: Points/Game,17.4,20.8
1,Scoring: Games - Points,8 - 139,8 - 166
2,First Downs: Total,136,168
3,First Downs: Rushing - Passing - By Penalty,56 - 61 - 19,75 - 84 - 9
4,Rushing: Yards / Attempt,3.11,3.98
5,Rushing: Attempts - Yards - TD,301 - 935 - 6,323 - 1286 - 12
6,Passing: Rating,115.96,117.16
7,Passing: Yards,1416,1703
8,Passing: Attempts - Completions - Interceptio...,219 - 114 - 6 - 10,269 - 155 - 3 - 7
9,Total Offense: Yards / Play,4.52,5.05


In [41]:
df_cal = df_cal.drop(columns=['Opponents']).T
header = df_cal.iloc[0]
df_cal = df_cal[1:]
df_cal = df_cal.rename(columns = header)
df_cal

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
California,17.4,8 - 139,136,56 - 61 - 19,3.11,301 - 935 - 6,115.96,1416,219 - 114 - 6 - 10,4.52,...,33.33%,12 - 4,82.35%,17 - 14,69.2%,13 - 9,100%,16 - 16,-,0 - 0


# Combining the Team Stats DataFrames

Now that we have DataFrames for each team, we need to combine them into a single DataFrame.

In [42]:
df_teams = pd.concat([df_arz, df_azst, df_cal, df_col, df_ore, df_orst, df_stan, df_ucla, df_usc, df_utah, df_wash, df_wash_st])

In [43]:
df_teams

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
Arizona,32.3,8 - 258,189,82 - 101 - 6,5.24,309 - 1620 - 15,147.45,2316,285 - 181 - 9 - 19,6.63,...,81.82%,11 - 9,88%,25 - 22,60%,10 - 6,97%,33 - 32,100%,1 - 1
Arizona State,23.4,8 - 187,138,53 - 77 - 8,3.69,280 - 1033 - 10,146.67,1902,223 - 135 - 2 - 11,5.83,...,50%,10 - 5,92%,25 - 23,76.5%,17 - 13,100%,20 - 20,100%,1 - 1
California,17.4,8 - 139,136,56 - 61 - 19,3.11,301 - 935 - 6,115.96,1416,219 - 114 - 6 - 10,4.52,...,33.33%,12 - 4,82.35%,17 - 14,69.2%,13 - 9,100%,16 - 16,-,0 - 0
Colorado,27.1,8 - 217,175,73 - 90 - 12,4.31,296 - 1277 - 11,133.69,2097,289 - 182 - 9 - 14,5.77,...,61.54%,13 - 8,80.65%,31 - 25,80%,15 - 12,96.2%,26 - 25,-,0 - 0
Oregon,36.0,8 - 288,193,72 - 107 - 14,5.05,301 - 1521 - 14,162.43,2207,276 - 189 - 1 - 23,6.46,...,50%,18 - 9,80.49%,41 - 33,62.5%,8 - 5,97.4%,38 - 37,100%,1 - 1
Oregon State,30.4,7 - 213,160,69 - 82 - 9,4.88,240 - 1170 - 12,138.28,1706,250 - 147 - 2 - 18,5.87,...,71.43%,14 - 10,88.46%,26 - 23,28.6%,7 - 2,96.4%,28 - 27,0%,2 - 0
Stanford,22.6,8 - 181,157,51 - 88 - 18,3.68,259 - 954 - 6,125.57,1834,283 - 171 - 3 - 11,5.14,...,30%,10 - 3,92.31%,26 - 24,77.8%,18 - 14,100%,19 - 19,0%,1 - 0
UCLA,29.1,8 - 233,176,77 - 83 - 16,4.02,333 - 1337 - 13,134.34,1895,265 - 158 - 7 - 16,5.40,...,42.86%,21 - 9,81.82%,33 - 27,62.5%,8 - 5,100%,30 - 30,100%,1 - 1
USC,31.3,8 - 250,175,64 - 102 - 9,4.48,261 - 1170 - 13,155.46,2369,282 - 200 - 10 - 18,6.52,...,66.67%,6 - 4,77.42%,31 - 24,88.9%,9 - 8,100%,32 - 32,-,0 - 0
Utah,33.1,8 - 265,185,92 - 75 - 18,5.07,346 - 1755 - 21,175.61,1864,185 - 134 - 1 - 11,6.82,...,64.29%,14 - 9,80.56%,36 - 29,63.6%,11 - 7,97.1%,35 - 34,-,0 - 0


We now just have a single DataFrame with all of the PAC-12 teams.  There is an issue, however.  When our data was scraped, the team names were imported as index values instead of column values.  We will need to correct for this in order to perform a join later on with our conference statistics (df_conf) DataFrame.  The easiest way to do this is to just add a new column with the individual team names.  We will label this column "Name" so that it matches up with the column name in the df_conf DataFrame.

In [44]:
df_teams = df_teams.assign(Name = ["Arizona", "Arizona State", "California", "Colorado", "Oregon", "Oregon State", "Stanford", "UCLA", "USC", "Utah", "Washington", "Washington State"])
df_teams

,Scoring: Points/Game,Scoring: Games - Points,First Downs: Total,First Downs: Rushing - Passing - By Penalty,Rushing: Yards / Attempt,Rushing: Attempts - Yards - TD,Passing: Rating,Passing: Yards,Passing: Attempts - Completions - Interceptions - TD,Total Offense: Yards / Play,...,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made,Name
Arizona,32.3,8 - 258,189,82 - 101 - 6,5.24,309 - 1620 - 15,147.45,2316,285 - 181 - 9 - 19,6.63,...,11 - 9,88%,25 - 22,60%,10 - 6,97%,33 - 32,100%,1 - 1,Arizona
Arizona State,23.4,8 - 187,138,53 - 77 - 8,3.69,280 - 1033 - 10,146.67,1902,223 - 135 - 2 - 11,5.83,...,10 - 5,92%,25 - 23,76.5%,17 - 13,100%,20 - 20,100%,1 - 1,Arizona State
California,17.4,8 - 139,136,56 - 61 - 19,3.11,301 - 935 - 6,115.96,1416,219 - 114 - 6 - 10,4.52,...,12 - 4,82.35%,17 - 14,69.2%,13 - 9,100%,16 - 16,-,0 - 0,California
Colorado,27.1,8 - 217,175,73 - 90 - 12,4.31,296 - 1277 - 11,133.69,2097,289 - 182 - 9 - 14,5.77,...,13 - 8,80.65%,31 - 25,80%,15 - 12,96.2%,26 - 25,-,0 - 0,Colorado
Oregon,36.0,8 - 288,193,72 - 107 - 14,5.05,301 - 1521 - 14,162.43,2207,276 - 189 - 1 - 23,6.46,...,18 - 9,80.49%,41 - 33,62.5%,8 - 5,97.4%,38 - 37,100%,1 - 1,Oregon
Oregon State,30.4,7 - 213,160,69 - 82 - 9,4.88,240 - 1170 - 12,138.28,1706,250 - 147 - 2 - 18,5.87,...,14 - 10,88.46%,26 - 23,28.6%,7 - 2,96.4%,28 - 27,0%,2 - 0,Oregon State
Stanford,22.6,8 - 181,157,51 - 88 - 18,3.68,259 - 954 - 6,125.57,1834,283 - 171 - 3 - 11,5.14,...,10 - 3,92.31%,26 - 24,77.8%,18 - 14,100%,19 - 19,0%,1 - 0,Stanford
UCLA,29.1,8 - 233,176,77 - 83 - 16,4.02,333 - 1337 - 13,134.34,1895,265 - 158 - 7 - 16,5.40,...,21 - 9,81.82%,33 - 27,62.5%,8 - 5,100%,30 - 30,100%,1 - 1,UCLA
USC,31.3,8 - 250,175,64 - 102 - 9,4.48,261 - 1170 - 13,155.46,2369,282 - 200 - 10 - 18,6.52,...,6 - 4,77.42%,31 - 24,88.9%,9 - 8,100%,32 - 32,-,0 - 0,USC
Utah,33.1,8 - 265,185,92 - 75 - 18,5.07,346 - 1755 - 21,175.61,1864,185 - 134 - 1 - 11,6.82,...,14 - 9,80.56%,36 - 29,63.6%,11 - 7,97.1%,35 - 34,-,0 - 0,Utah


The new "Name" column has been added to the end of our DataFrame.  

We now need to join the two DataFrames.  We will use an inner join.  Since both DataFrames only have a single duplicate column, the join will automatically use this column ("Name").

In [45]:
df_all = pd.merge(df_conf, df_teams, how='inner')
df_all

,Name,G,TD,FG,1XP,2XP,Safety,Points,Points/G,Scoring: Points/Game,...,4th Down Conversions: Conversion %,4th Down Conversions: Attempts - Conversions,Red Zone: Success %,Red Zone: Attempts - Scores,Field Goals: Success %,Field Goals: Attempts - Made,PAT Kicking: Success %,PAT Kicking: Attempts - Made,2-Point Conversions: Success %,2-Point Conversions: Attempts - Made
0,Washington State,8,42,13,41,1,0,334,41.8,41.8,...,72.73%,11 - 8,92.68%,41 - 38,100%,13 - 13,100%,41 - 41,100%,1 - 1
1,Oregon,8,39,5,37,1,0,288,36.0,36.0,...,50%,18 - 9,80.49%,41 - 33,62.5%,8 - 5,97.4%,38 - 37,100%,1 - 1
2,Washington,8,34,15,33,1,1,286,35.8,35.8,...,62.5%,16 - 10,91.43%,35 - 32,100%,15 - 15,100%,33 - 33,100%,1 - 1
3,Utah,8,35,7,34,0,0,265,33.1,33.1,...,64.29%,14 - 9,80.56%,36 - 29,63.6%,11 - 7,97.1%,35 - 34,-,0 - 0
4,Arizona,8,34,6,32,1,1,258,32.3,32.3,...,81.82%,11 - 9,88%,25 - 22,60%,10 - 6,97%,33 - 32,100%,1 - 1
5,USC,8,32,8,32,0,1,250,31.3,31.3,...,66.67%,6 - 4,77.42%,31 - 24,88.9%,9 - 8,100%,32 - 32,-,0 - 0
6,Oregon State,7,30,2,27,0,0,213,30.4,30.4,...,71.43%,14 - 10,88.46%,26 - 23,28.6%,7 - 2,96.4%,28 - 27,0%,2 - 0
7,UCLA,8,31,5,30,1,0,233,29.1,29.1,...,42.86%,21 - 9,81.82%,33 - 27,62.5%,8 - 5,100%,30 - 30,100%,1 - 1
8,Colorado,8,26,12,25,0,0,217,27.1,27.1,...,61.54%,13 - 8,80.65%,31 - 25,80%,15 - 12,96.2%,26 - 25,-,0 - 0
9,Arizona State,8,21,13,20,1,0,187,23.4,23.4,...,50%,10 - 5,92%,25 - 23,76.5%,17 - 13,100%,20 - 20,100%,1 - 1


Our exercise is now complete.  In summary, here are the data transform/clean-up steps we performed to get to this final DataFrame:  
* Two separate data scrapes  
* Dropped unnecessary columns  
* Performed a rows to columns transform  
* Created column header names  
* Added a new column  
* Merged two DataFrames  
* Performed an inner join

The above stats summary is for Week 10 in the Pac-12.  

For comparison sake, and to demonstrate the power of this kind of web scraping, I have included the scraping results for weeks 11 and 12.  To generate these, all I had to do was re-run the code.  The scraping took care of everything in just a matter of seconds.

In [1]:
from IPython.display import Image
img = 'week_11.png'
Image(url=img)

In [2]:
img = 'week_12.png'
Image(url=img)